In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("train.txt", delimiter=';', header=None, names=['sentence','label'])
df_test = pd.read_csv("test.txt", delimiter=';', header=None, names=['sentence','label'])
df_val = pd.read_csv("val.txt", delimiter=';', header=None, names=['sentence','label'])

In [3]:
df = pd.concat([df_train,df_val])

In [4]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['label_encoded'] = labelencoder.fit_transform(df['label'])


In [5]:
df

,sentence,label,label_encoded
0,i didnt feel humiliated,sadness,4
1,i can go from feeling so hopeless to so damned...,sadness,4
2,im grabbing a minute to post i feel greedy wrong,anger,0
3,i am ever feeling nostalgic about the fireplac...,love,3
4,i am feeling grouchy,anger,0
...,...,...,...
1995,im having ssa examination tomorrow in the morn...,sadness,4
1996,i constantly worry about their fight against n...,joy,2
1997,i feel its important to share this info for th...,joy,2
1998,i truly feel that if you are passionate enough...,joy,2


In [6]:
df[['label','label_encoded']].drop_duplicates(keep='first')

,label,label_encoded
0,sadness,4
2,anger,0
3,love,3
6,surprise,5
7,fear,1
8,joy,2


In [7]:
df.dtypes

sentence         object
label            object
label_encoded     int32
dtype: object

In [8]:
seqlen = df['sentence'].apply(lambda x: len(x.split()))

In [9]:
df["label"].value_counts()

joy         6066
sadness     5216
anger       2434
fear        2149
love        1482
surprise     653
Name: label, dtype: int64

In [10]:
max_seqlen = 50

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [12]:
import numpy as np
import pandas as pd

In [13]:
X_ids = np.zeros((len(df), max_seqlen))
X_mask = np.zeros((len(df), max_seqlen))

In [14]:
for i, sequence in enumerate(df['sentence']):
    token = tokenizer.encode_plus(sequence, 
                              add_special_tokens=True,
                              max_length=max_seqlen, 
                              truncation=True,
                              padding="max_length",
                              return_token_type_ids=False,
                              return_attention_mask=True,
                              return_tensors='tf')
    X_ids[i,:], X_mask[i,: ] = token['input_ids'],token['attention_mask']

In [15]:
X_ids

array([[  101.,   178.,  1238., ...,     0.,     0.,     0.],
       [  101.,   178.,  1169., ...,     0.,     0.,     0.],
       [  101., 13280., 10810., ...,     0.,     0.,     0.],
       ...,
       [  101.,   178.,  1631., ...,     0.,     0.,     0.],
       [  101.,   178.,  5098., ...,     0.,     0.,     0.],
       [  101.,   178.,  1631., ...,     0.,     0.,     0.]])

In [16]:
X_mask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [17]:
arr=df['label_encoded'].values
arr.size

18000

In [18]:
arr.max()

5

In [19]:
labels = np.zeros((arr.size,arr.max()+1))
labels

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [20]:
labels[np.arange(arr.size),arr] = 1
labels

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [21]:
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((X_ids,X_mask,labels))   

In [22]:
for i in dataset.take(1):
    print(i)

(<tf.Tensor: shape=(50,), dtype=float64, numpy=
array([  101.,   178.,  1238.,  1204.,  1631., 21820., 21896.,   102.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.])>, <tf.Tensor: shape=(50,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>, <tf.Tensor: shape=(6,), dtype=float64, numpy=array([0., 0., 0., 0., 1., 0.])>)


In [23]:
def map_function(input_ids,attention_mask,labels):
    return{'input_ids':input_ids,'attention_mask':attention_mask},labels

In [24]:
dataset = dataset.map(map_function)

In [25]:
dataset

<MapDataset element_spec=({'input_ids': TensorSpec(shape=(50,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(50,), dtype=tf.float64, name=None)}, TensorSpec(shape=(6,), dtype=tf.float64, name=None))>

In [26]:
for i in dataset.take(1):
    print(i)

({'input_ids': <tf.Tensor: shape=(50,), dtype=float64, numpy=
array([  101.,   178.,  1238.,  1204.,  1631., 21820., 21896.,   102.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.])>, 'attention_mask': <tf.Tensor: shape=(50,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>}, <tf.Tensor: shape=(6,), dtype=float64, numpy=array([0., 0., 0., 0., 1., 0.])>)


In [27]:
dataset = dataset.shuffle(10000).batch(32)

In [28]:
dataset

<BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 50), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(None, 50), dtype=tf.float64, name=None)}, TensorSpec(shape=(None, 6), dtype=tf.float64, name=None))>

In [29]:
no_of_batches = len(dataset)

In [30]:
train = dataset.take(round(no_of_batches * 0.9))
validation = dataset.skip(round(no_of_batches * 0.9))

In [31]:
from transformers import TFAutoModel
bert = TFAutoModel.from_pretrained("bert-base-cased")

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [32]:
input_ids = tf.keras.layers.Input(shape=(max_seqlen,), name="input_ids",dtype='int32')
mask = tf.keras.layers.Input(shape=(max_seqlen,), name="attention_mask",dtype='int32')

embedding = bert(input_ids, attention_mask=mask)[0]
X = tf.keras.layers.GlobalMaxPooling1D()(embedding)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(128, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
y = tf.keras.layers.Dense(6, activation='softmax',name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)


In [33]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 50,                                            

In [34]:
model.layers[2].trainable = False

In [35]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 50)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 50,                                            

In [36]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
history = model.fit(
    train,
    validation_data=validation,
    epochs=100)

Epoch 1/100


507/507 [==============================] - 162s 266ms/step - loss: 1.4054 - accuracy: 0.4800 - val_loss: 1.0437 - val_accuracy: 0.6002
Epoch 2/100
507/507 [==============================] - 129s 254ms/step - loss: 1.1910 - accuracy: 0.5486 - val_loss: 0.9716 - val_accuracy: 0.6425
Epoch 3/100
507/507 [==============================] - 130s 257ms/step - loss: 1.1297 - accuracy: 0.5713 - val_loss: 0.9147 - val_accuracy: 0.6577
Epoch 4/100
507/507 [==============================] - 128s 253ms/step - loss: 1.1031 - accuracy: 0.5812 - val_loss: 0.8907 - val_accuracy: 0.6695
Epoch 5/100
507/507 [==============================] - 143s 281ms/step - loss: 1.0815 - accuracy: 0.5912 - val_loss: 0.8843 - val_accuracy: 0.6661
Epoch 6/100
507/507 [==============================] - 149s 293ms/step - loss: 1.0704 - accuracy: 0.5912 - val_loss: 0.8401 - val_accuracy: 0.6914
Epoch 7/100
507/507 [==============================] - 151s 299ms/step - loss: 1.0494 - accuracy: 0.5992 - val_loss: 0.8572 - val_

In [38]:
model.save("sentiment_model.h5")

In [39]:
model.save("sentiment_model")

INFO:tensorflow:Assets written to: sentiment_model\assets


INFO:tensorflow:Assets written to: sentiment_model\assets
